In [1]:
import pandas as pd
from IPython.display import display
import datetime as dt
from matplotlib import pylab
import numpy as np
import scipy as sp
import statsmodels.api as sm
import warnings 
warnings.filterwarnings("ignore")
import sqlite3

In [2]:
batting_data = "./data/csv_data/core/Batting.csv"
people_data = "./data/csv_data/core/People.csv"
fielding_data = "./data/csv_data/core/Fielding.csv"
franchise_data = "./data/csv_data/core/TeamsFranchises.csv"
teams_data = "./data/csv_data/core/Teams.csv"
appearances_data = "./data/csv_data/core/Appearances.csv"
park_data = "./data/csv_data/core/Parks.csv"

# include all batting columns
# include all appearances columns
# include all parks columns

people_cols = ['playerID', 'birthYear', 'birthMonth', 'nameFirst', 'nameLast', 'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame', 'retroID', 'bbrefID']

fielding_cols = ['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS']

team_cols = ['yearID', 'lgID', 'teamID', 'franchID', 'divID', 'Rank', 'G', 'Ghome', 'W', 'L', 'park', 'name']

franch_cols = ['franchID', 'franchName', 'active']


df_batting = pd.read_csv(batting_data, header=0)
df_app = pd.read_csv(appearances_data, header=0)
df_parks = pd.read_csv(park_data, header=0)
df_people = pd.read_csv(people_data, header=0, usecols=people_cols)
df_fielding = pd.read_csv(fielding_data, header=0, usecols=fielding_cols)
df_teams = pd.read_csv(teams_data, header=0, usecols=team_cols)
df_franch = pd.read_csv(franchise_data, header=0, usecols=franch_cols)

display(df_batting.head(5))
display(df_app.head(5))
display(df_parks.head(5))
display(df_people.head(5))
display(df_fielding.head(5))
display(df_teams.head(5))
display(df_franch.head(5))

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
0,1871,TRO,NaN,abercda01,1,1.0,1,1.0,0,0,...,0,0,1,0,0,0,0,0.0,0.0,0.0
1,1871,RC1,NaN,addybo01,25,25.0,25,25.0,0,0,...,22,0,3,0,0,0,0,0.0,0.0,0.0
2,1871,CL1,NaN,allisar01,29,29.0,29,29.0,0,0,...,2,0,0,0,29,0,29,0.0,0.0,0.0
3,1871,WS3,NaN,allisdo01,27,27.0,27,27.0,0,27,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,1871,RC1,NaN,ansonca01,25,25.0,25,25.0,0,5,...,2,20,0,1,0,0,1,0.0,0.0,0.0


,park.key,park.name,park.alias,city,state,country
0,ALB01,Riverside Park,NaN,Albany,NY,US
1,ALT01,Columbia Park,NaN,Altoona,PA,US
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,US
3,ARL01,Arlington Stadium,NaN,Arlington,TX,US
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,US


,playerID,birthYear,birthMonth,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,Hank,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,Tommie,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,Don,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,Andy,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


,playerID,yearID,stint,teamID,lgID,POS,G,GS
0,abercda01,1871,1,TRO,NaN,SS,1,1.0
1,addybo01,1871,1,RC1,NaN,2B,22,22.0
2,addybo01,1871,1,RC1,NaN,SS,3,3.0
3,allisar01,1871,1,CL1,NaN,2B,2,0.0
4,allisar01,1871,1,CL1,NaN,OF,29,29.0


,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,name,park
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,Boston Red Stockings,South End Grounds I
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,Chicago White Stockings,Union Base-Ball Grounds
2,1871,NaN,CL1,CFC,NaN,8,29,NaN,10,19,Cleveland Forest Citys,National Association Grounds
3,1871,NaN,FW1,KEK,NaN,7,19,NaN,7,12,Fort Wayne Kekiongas,Hamilton Field
4,1871,NaN,NY2,NNA,NaN,5,33,NaN,16,17,New York Mutuals,Union Grounds (Brooklyn)


,franchID,franchName,active
0,ALT,Altoona Mountain City,N
1,ANA,Los Angeles Angels of Anaheim,Y
2,ARI,Arizona Diamondbacks,Y
3,ATH,Philadelphia Athletics,N
4,ATL,Atlanta Braves,Y


In order to filter/merge the ballparks based on teams parks needs to be extended in order to account for alias name references.

In [3]:
df_park_split = df_parks['park.alias'].str.split(";", n=5, expand=True)
#display(df_park_split)

df_parks2 = df_parks.merge(df_park_split, how='left', left_index=True, right_index=True)
#display(df_parks2.head(5))

df_park_extend_0 = df_parks2[['park.key', 0, 'city', 'state', 'country']]
df_park_extend_1 = df_parks2[['park.key', 1, 'city', 'state', 'country']]
df_park_extend_2 = df_parks2[['park.key', 2, 'city', 'state', 'country']]
df_park_extend_3 = df_parks2[['park.key', 3, 'city', 'state', 'country']]

df_park_extend_0.rename(columns={0: 'park.name'}, inplace=True)
df_park_extend_1.rename(columns={1: 'park.name'}, inplace=True)
df_park_extend_2.rename(columns={2: 'park.name'}, inplace=True)
df_park_extend_3.rename(columns={3: 'park.name'}, inplace=True)

df_park_extend_0.dropna(subset=['park.name'], inplace=True)
df_park_extend_1.dropna(subset=['park.name'], inplace=True)
df_park_extend_2.dropna(subset=['park.name'], inplace=True)
df_park_extend_3.dropna(subset=['park.name'], inplace=True)

print(f"Number of parks not including alias names: {len(df_parks.index)}")

df_parks = df_parks.append(df_park_extend_0, ignore_index=True)
df_parks = df_parks.append(df_park_extend_1, ignore_index=True)
df_parks = df_parks.append(df_park_extend_2, ignore_index=True)
df_parks = df_parks.append(df_park_extend_3, ignore_index=True)

df_parks.drop_duplicates(inplace=True, ignore_index=True)

print(f"Number of parks including alias names {len(df_parks.index)}")
#display(df_parks.head(5))
#display(df_parks.tail(5))

Number of parks not including alias names: 255
Number of parks including alias names 326


Initial Constrains:
- Minimum number of at bats = 150
- Exclude Pitchers
    - players that are pitchers the majority of their games played
- Minimum debut year = 1970
    - we want the stats of a player over his full career and stats prior to this year may be missing and/or inaccurate
    - drop players with missing debut year


In [4]:
df_people = df_people.dropna(axis=0, subset=['debut'])

def get_year(cell):

    if isinstance(cell, str):
        debut_dt = dt.datetime.strptime(cell, "%Y-%m-%d")
        return debut_dt.year 
    
    return np.nan

def get_month(cell):

    if isinstance(cell, str):
        final_dt = dt.datetime.strptime(cell, "%Y-%m-%d")
        return final_dt.month
    
    return np.nan

df_people['debut_yr'] = df_people['debut'].apply(get_year)

print(len(df_people.index))
df_people = df_people[df_people['debut_yr'] > 1970]
print(len(df_people.index))

df_people['final_month'] = df_people['finalGame'].apply(get_month)
df_people['final_yr'] = df_people['finalGame'].apply(get_year)
df_people['final_yr_adj'] = np.where(df_people['final_month'] < 5, df_people['final_yr'] - 1, df_people['final_yr'])
df_people['seasons_exp'] = df_people['final_yr_adj'] - df_people['debut_yr']

print(len(df_people.index)) # shouldn't change

19892
9193
9193


Get actual number of seasons played and check if a player missed any seasons.
This captures any missing seasons, regardless of reason (injuries, time off, or our exclusion critieria)

In [5]:
batting_seasons = df_batting[['playerID', 'yearID']]
season_count = batting_seasons.groupby(['playerID'])['yearID'].count()
season_count = season_count.to_frame().reset_index().rename(columns={'yearID': 'seasons_act'})
display(season_count.head())

df_people = df_people.merge(season_count, how='left', on='playerID')
df_people.dropna(subset=['seasons_act'], inplace=True) # don't need to do this if I do inner merge
df_people['missed_any'] = np.where(df_people['seasons_exp'] != df_people['seasons_act'], 1, 0)

print(len(df_people.index))
display(df_people.head())

,playerID,seasons_act
0,aardsda01,9
1,aaronha01,23
2,aaronto01,7
3,aasedo01,13
4,abadan01,3


9192


,playerID,birthYear,birthMonth,nameFirst,nameLast,nameGiven,weight,height,bats,throws,...,finalGame,retroID,bbrefID,debut_yr,final_month,final_yr,final_yr_adj,seasons_exp,seasons_act,missed_any
0,aardsda01,1981.0,12.0,David,Aardsma,David Allan,215.0,75.0,R,R,...,2015-08-23,aardd001,aardsda01,2004,8,2015,2015,11,9.0,1
1,aasedo01,1954.0,9.0,Don,Aase,Donald William,190.0,75.0,R,R,...,1990-10-03,aased001,aasedo01,1977,10,1990,1990,13,13.0,0
2,abadan01,1972.0,8.0,Andy,Abad,Fausto Andres,184.0,73.0,L,L,...,2006-04-13,abada001,abadan01,2001,4,2006,2005,4,3.0,1
3,abadfe01,1985.0,12.0,Fernando,Abad,Fernando Antonio,235.0,74.0,L,L,...,2019-09-28,abadf001,abadfe01,2010,9,2019,2019,9,10.0,1
4,abbotgl01,1951.0,2.0,Glenn,Abbott,William Glenn,200.0,78.0,R,R,...,1984-08-08,abbog001,abbotgl01,1973,8,1984,1984,11,12.0,1


Identify player's primary position and filter out pitchers.

In [6]:
# identify player by primary position
df_pos = df_fielding.groupby(['playerID', 'POS'])['G'].count()
df_pos = df_pos.unstack()

pos_cols = ['1B', '2B', '3B', 'C', 'OF', 'P', 'SS']

def get_main_pos(row):
    # return col name for max pos
    max_col_val = 0
    max_col = ''
    for col in pos_cols:
        if row[col] > max_col_val:
            max_col = col

    return max_col

df_pos['p_pos'] = df_pos.apply(get_main_pos, axis=1) 
df_pos.reset_index(inplace=True)
df_pos = df_pos[['playerID', 'p_pos']]
df_pos = df_pos.rename_axis(None, axis=1)
#display(df_pos.head(5))
print(len(df_people.index))
df_people = df_people.merge(df_pos, how='inner', on='playerID')
df_people = df_people[df_people['p_pos'] != 'P']
print(len(df_people.index))

9192
4096


In [7]:
debut_filter = df_people[['playerID']]

print(f"Batting start count: {len(df_batting.index)}")
print(f"Appearance start cound: {len(df_app.index)}")
print(f"Parks start count: {len(df_parks.index)}")
print(f"Fielding start count: {len(df_fielding.index)}")
print(f"Teams start count: {len(df_teams.index)}")
print(f"Franchise start count: {len(df_franch.index)}")

# Filter the remaining dataframes to include restricted player sample
df_batting = df_batting.merge(debut_filter, how='inner', on='playerID') 
df_app = df_app.merge(debut_filter, how='inner', on='playerID') 
df_fielding = df_fielding.merge(debut_filter, how='inner', on='playerID')

# Teams can be filtered by yearID
df_teams = df_teams[df_teams['yearID'] >= 1970]

# Parks can be filtered by remaining parks from filtered teams
park_filter = df_teams[['park']]
df_parks = df_parks.merge(park_filter, how='inner', left_on='park.name', right_on='park')

# Franchises can be filtered by remaining franchID from filtered teams
franch_filter = df_teams[['franchID']]
df_franch = df_franch.merge(franch_filter, how='inner', on='franchID')

df_batting.drop_duplicates(inplace=True, ignore_index=True)
df_app.drop_duplicates(inplace=True, ignore_index=True)
df_fielding.drop_duplicates(inplace=True, ignore_index=True)
df_teams.drop_duplicates(inplace=True, ignore_index=True)
df_parks.drop_duplicates(inplace=True, ignore_index=True)
df_franch.drop_duplicates(inplace=True, ignore_index=True)


print("\n ---------- \n")
print(f"Batting end count: {len(df_batting.index)}")
print(f"Appearance end count: {len(df_app.index)}")
print(f"Parks end count: {len(df_parks.index)}")
print(f"Fielding end count: {len(df_fielding.index)}")
print(f"Teams end count: {len(df_teams.index)}")
print(f"Franchise end count: {len(df_franch.index)}")

Batting start count: 108789
Appearance start cound: 108717
Parks start count: 326
Fielding start count: 144768
Teams start count: 2955
Franchise start count: 120

 ---------- 

Batting end count: 26665
Appearance end count: 26652
Parks end count: 61
Fielding end count: 42100
Teams end count: 1414
Franchise end count: 30


In [8]:
# Only include instances where a player had at least the minimum number of at bats
ab_min = 150
df_batting = df_batting[df_batting.AB >= ab_min]
print(f"Final count of batting instances: {len(df_batting.index)}")

Final count of batting instances: 14224


Calculate TB, OBP, SLG, and OPS

In [9]:
df_batting['1B'] = df_batting['H'] - (df_batting['2B'] + df_batting['3B'] + df_batting['HR'])
df_batting['TB'] = df_batting['1B'] + (df_batting['2B'] * 2) + (df_batting['3B'] * 3) + (df_batting['HR'] * 4)
df_batting['obp'] = (df_batting.H + df_batting.BB + df_batting.HBP) / (df_batting.AB + df_batting.BB + df_batting.SF + df_batting.HBP)
df_batting['slg'] = df_batting.TB / df_batting.AB
df_batting['ops'] = df_batting['obp'] + df_batting['slg']

Identify if a player bats and throws with the same hand (bt_same)

In [10]:
df_people['bt_same'] = np.where(df_people['bats'] == df_people['throws'], 1, 0)

Calculate Players Age and years of experience (yoe)

In [11]:
df_birth_years = df_people[['playerID','birthYear', 'final_yr_adj', 'debut_yr', 'missed_any', 'bt_same', 'p_pos', 'height', 'weight']]

df_main = df_batting.merge(df_birth_years, how='inner', on='playerID')
df_main['age'] = df_main['yearID'] - df_main['birthYear']
df_main['yoe'] = df_main['yearID'] - df_main['debut_yr']

df_main.drop(columns=['birthYear'], inplace=True)
print(len(df_main.index))
display(df_main.head())

14224


,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,ops,final_yr_adj,debut_yr,missed_any,bt_same,p_pos,height,weight,age,yoe
0,arnolch01,1974,1,SFN,NL,78,174,22,42,7,...,0.635417,1976,1971,1,1,SS,70.0,160.0,27.0,3
1,auerbri01,1971,1,ML4,AL,79,236,22,48,10,...,0.529792,1981,1971,1,1,SS,72.0,165.0,21.0,0
2,auerbri01,1972,1,ML4,AL,153,554,50,121,16,...,0.545620,1981,1971,1,1,SS,72.0,165.0,22.0,1
3,auerbri01,1975,1,LAN,NL,85,170,18,38,9,...,0.574343,1981,1971,1,1,SS,72.0,165.0,25.0,4
4,beniqju01,1974,1,BOS,AL,106,389,60,104,14,...,0.669826,1988,1971,1,1,SS,71.0,150.0,24.0,3


Calculate adjusted years of experience (yoe_adj).
yoe_adj is the numbers of years a player has been included in our data.
i.e. only count seasons they played and had at least 150 at bats (AB).

In [12]:
df_start_end = df_main[['playerID', 'yearID', 'debut_yr', 'final_yr_adj', 'yoe']]
df_start_end.sort_values(by=['yearID'], inplace=True)


player_careers = {}
count = 0

for ind, row in df_start_end.iterrows():

    if row['playerID'] not in player_careers:
        player_careers[row['playerID']] = {}

    player_careers[row['playerID']]['exp_career'] = [x for x in range(row['debut_yr'], row['final_yr_adj'] + 1)]
    
    if 'act_career' not in player_careers[row['playerID']].keys():
        player_careers[row['playerID']]['act_career'] = []

    player_careers[row['playerID']]['act_career'].append(row['yearID'])

missed_years = {}
for player in player_careers:
    exp_years = set(player_careers[player]['exp_career'])
    act_years = set(player_careers[player]['act_career'])

    if exp_years != act_years:
        missed_years[player] = [int(x) for x in exp_years.difference(act_years)]



df_sub_main = df_main[['playerID','yearID','yoe']]

for index, row in df_main.iterrows():
    player = row['playerID']
    yearID = row['yearID']
    yoe_adj = row['yoe']

    if player in missed_years:
        
        for year in missed_years[player]:
            if yearID > year:
                yoe_adj -= 1
    
    df_sub_main['yoe'].iloc[index] = yoe_adj


df_sub_main.rename(columns={'yoe': 'yoe_adj'}, inplace=True)
df_main = df_main.merge(df_sub_main, how='inner', on=['playerID','yearID'])

#print(missed_years['auerbri01'])
#display(df_sub_main[df_sub_main['playerID'] == 'auerbri01'])
#display(df_main[df_main['playerID'] == 'auerbri01'])

classify breakout seasons and classify if a player ever has a breakout season.

Defining breakout players by approximately the 90th percentile in ops increase, which corresponds to approximately >= .175 increase in ops from one year to the next.

In [13]:
#df_main.sort_values(by=['playerID', 'yearID'], inplace=True)
#df_main['ops_per_delta'] = df_main['ops'].pct_change(periods=1, fill_method='ffill')
#df_main.dropna(subset=['ops_per_delta'], inplace=True)

df_main['ops_per_delta'] = df_main.groupby(['playerID'])['ops'].apply(lambda x: pd.Series.pct_change(x, periods=1, fill_method='ffill'))
df_main.dropna(subset=['ops_per_delta'], inplace=True)
display(df_main.head())

df_post_debut = df_main[df_main['yoe_adj'] > 0]
#df_post_debut['ops_per_delta'].describe([.5, .75, .8, .9, .95])
df_main['bo_season'] = np.where( (df_main['ops_per_delta'] >= .175) & (df_main['yoe_adj'] > 0), 1, 0)
#display(df_main[df_main['bo_season'] == 1]) #1178 instance

breakout_players = df_main[df_main['bo_season'] == 1]
breakout_players = breakout_players[['playerID', 'bo_season']]
breakout_players.rename(columns={'bo_season': 'bo_ever'}, inplace=True)

df_main = df_main.merge(breakout_players, how='left', on='playerID')
df_main['bo_ever'].fillna(0, inplace=True)
display(df_main.head())

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,debut_yr,missed_any,bt_same,p_pos,height,weight,age,yoe,yoe_adj,ops_per_delta
2,auerbri01,1972,1,ML4,AL,153,554,50,121,16,...,1971,1,1,SS,72.0,165.0,22.0,1,1,0.029875
3,auerbri01,1975,1,LAN,NL,85,170,18,38,9,...,1971,1,1,SS,72.0,165.0,25.0,4,2,0.052643
5,beniqju01,1975,1,BOS,AL,78,254,43,74,14,...,1971,1,1,SS,71.0,150.0,25.0,4,1,0.134220
6,beniqju01,1976,1,TEX,AL,145,478,49,122,14,...,1971,1,1,SS,71.0,150.0,26.0,5,2,-0.189141
7,beniqju01,1977,1,TEX,AL,123,424,56,114,19,...,1971,1,1,SS,71.0,150.0,27.0,6,3,0.215689


,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,bt_same,p_pos,height,weight,age,yoe,yoe_adj,ops_per_delta,bo_season,bo_ever
0,auerbri01,1972,1,ML4,AL,153,554,50,121,16,...,1,SS,72.0,165.0,22.0,1,1,0.029875,0,0.0
1,auerbri01,1975,1,LAN,NL,85,170,18,38,9,...,1,SS,72.0,165.0,25.0,4,2,0.052643,0,0.0
2,beniqju01,1975,1,BOS,AL,78,254,43,74,14,...,1,SS,71.0,150.0,25.0,4,1,0.134220,0,1.0
3,beniqju01,1975,1,BOS,AL,78,254,43,74,14,...,1,SS,71.0,150.0,25.0,4,1,0.134220,0,1.0
4,beniqju01,1976,1,TEX,AL,145,478,49,122,14,...,1,SS,71.0,150.0,26.0,5,2,-0.189141,0,1.0


Add frachID and park names onto main table to be used in final output, not as a model factor of interest (at this time).

In [14]:
df_teams_subset = df_teams[['yearID', 'teamID', 'franchID', 'name', 'park']]
df_main = df_main.merge(df_teams_subset, on=['yearID', 'teamID'], how='inner')
df_main.drop_duplicates(subset=['playerID', 'yearID'], inplace=True)

print(f"Final length of df_main: {len(df_main.index)}")

Final length of df_main: 11593


In [15]:
df_main['bo_ever'] = df_main['bo_ever'].astype('int32')
df_main['age'] = df_main['age'].astype('int32')

df_main.dtypes

playerID          object
yearID             int64
stint              int64
teamID            object
lgID              object
G                  int64
AB                 int64
R                  int64
H                  int64
2B                 int64
3B                 int64
HR                 int64
RBI              float64
SB               float64
CS               float64
BB                 int64
SO               float64
IBB              float64
HBP              float64
SH               float64
SF               float64
GIDP             float64
1B                 int64
TB                 int64
obp              float64
slg              float64
ops              float64
final_yr_adj       int64
debut_yr           int64
missed_any         int32
bt_same            int32
p_pos             object
height           float64
weight           float64
age                int32
yoe                int64
yoe_adj            int64
ops_per_delta    float64
bo_season          int32
bo_ever            int32


Arbitrary Column formatting

In [16]:
# Batting cols float to int
batting_ints = {'RBI':'int32', 'SB':'int32', 'CS':'int32', 'SO':'int32', 'IBB':'int32', 'HBP':'int32', 'SH':'int32', 'SF':'int32', 'GIDP':'int32'}

app_ints = {'GS':'int32', 'G_defense':'int32', 'G_dh':'int32', 'G_ph':'int32', 'G_pr':'int32'}

fielding_ints = {'GS':'int32'}

# leaving weight and height as floats
people_ints = {'birthYear':'int32', 'birthMonth':'int32'}

team_ints = {'Ghome':'int32'}

df_main = df_main.astype(batting_ints)
df_batting = df_batting.astype(batting_ints)
df_app = df_app.astype(app_ints)
df_fielding = df_fielding.astype(fielding_ints)
df_people = df_people.astype(people_ints)
df_teams = df_teams.astype(team_ints)

Write to csv files

In [17]:
df_batting.to_csv("./data/cleaned_data/batting_clean.csv" , index=False)
df_app.to_csv("./data/cleaned_data/appearance_clean.csv" , index=False)
df_people.to_csv("./data/cleaned_data/people_clean.csv", index=False)
df_fielding.to_csv("./data/cleaned_data/fielding_clean.csv" , index=False)
df_teams.to_csv("./data/cleaned_data/teams_clean.csv", index=False)
df_parks.to_csv("./data/cleaned_data/parks_clean.csv", index=False)
df_franch.to_csv("./data/cleaned_data/franch_clean.csv", index=False)
df_main.to_csv("./data/cleaned_data/main_clean.csv", index=False)

Write to sqlite db

In [18]:
con = sqlite3.connect('./data/cleaned_data/cleaned_data.db')

df_batting.to_sql("batting_clean", con, index=False, if_exists='replace')
df_app.to_sql("appearance_clean", con,  index=False, if_exists='replace')
df_people.to_sql("people_clean", con, index=False, if_exists='replace')
df_fielding.to_sql("fielding_clean", con, index=False, if_exists='replace')
df_teams.to_sql("teams_clean", con, index=False, if_exists='replace')
df_parks.to_sql("parks_clean", con, index=False, if_exists='replace')
df_franch.to_sql("franch_clean", con, index=False, if_exists='replace')
df_main.to_sql("main_clean", con, index=False, if_exists='replace')

con.commit()
con.close()